In [1]:
from poker import Range
from treys import Card
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from cfr import CFRTrainer, MCCFRTrainer, ISet
from state import State
from exploitability import exploitability

In [2]:
board = [Card.new('Td'), Card.new('9d'), Card.new('6h'), Card.new('2h'), Card.new('2c')]

init_state = State()
init_state.set_board(board)

In [3]:
trainer = MCCFRTrainer(init_state, discount=True)

In [ ]:
# for i in range(5):
trainer.train(5000)
#     print(trainer._nodes_touched, exploitability(trainer))

In [ ]:
from state import build_range
from treys import Card

# p1_range = build_range(Range('22+ A2+ K2+ Q2+ J2+ T2+ 92+ 82+ 72+ 62+ 52+ 42+ 32+').combos)
# p2_range = build_range(Range('22+ A2+ K2+ Q2+ J2+ T2+ 92+ 82+ 72+ 62+ 52+ 42+ 32+').combos)
p1_range = build_range(Range('88+ AJo+ ATs+ KQ KJ JTs T9s').combos)
p2_range = build_range(Range('88+ AJo+ ATs+ KQ KJ JTs T9s').combos)

def get_ranks(combo) -> (int, int):
    combo = [Card.int_to_str(c) for c in combo]
    def rank_to_int(rank):
        rank = str(rank)
        if rank == 'A': return 12
        if rank == 'K': return 11
        if rank == 'Q': return 10
        if rank == 'J': return 9
        if rank == 'T': return 8
        return int(rank) - 2

    return (
        rank_to_int(combo[0][0]),
        rank_to_int(combo[1][0]))

def is_suited(combo):
    combo = [Card.int_to_str(c) for c in combo]
    return combo[0][1] == combo[1][1]

def print_strategy(player_idx, action_idx, history):
    # get infosets for initial betting round
    range_matrix = np.zeros((13, 13))
    weight_matrix = np.zeros((13, 13))
    x_axis_labels = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
    y_axis_labels = ['A', 'K', 'Q', 'J', 'T', '9', '8', '7', '6', '5', '4', '3', '2']
    range_sum = 0
    weight_sum = 0
    for k in trainer._infosets:
        player, card, hist = k.split(' ')
        if int(player) == player_idx and hist == history:
            strategy = trainer._infosets[k].get_final_strategy()
            
            rank1, rank2 = get_ranks(p1_range[int(card)] if player_idx == 0 else p2_range[int(card)])
            suited = is_suited(p1_range[int(card)]) if player_idx == 0 else is_suited(p2_range[int(card)])
                        
#             if player_idx == 0:
#                 print([Card.int_to_str(c) for c in p1_range[int(card)]],
#                       trainer._infosets[k].regrets,
#                       trainer._infosets[k].get_final_strategy())
#             if player_idx == 1 and rank1 == 3 and rank2 == 3:
#                 print([Card.int_to_str(c) for c in p2_range[int(card)]], trainer._infosets[k].regrets)
            
            range_sum += strategy[action_idx]
            weight_sum += 1
            
            if suited:
                range_matrix[12-rank1][12-rank2] += strategy[action_idx] # or 1
                weight_matrix[12-rank1][12-rank2] += 1
            else:
                range_matrix[12-rank2][12-rank1] += strategy[action_idx] # or 1
                weight_matrix[12-rank2][12-rank1] += 1
    # normalize
    for i in range(13):
        for j in range(13):
            if weight_matrix[i][j] > 0:
                range_matrix[i][j] /= weight_matrix[i][j]

    print("action %:", range_sum / weight_sum)
    plt.figure(figsize=(7,6))
    sns.heatmap(range_matrix, xticklabels=x_axis_labels, yticklabels=y_axis_labels)
    plt.show()
    

print([Card.int_to_str(c) for c in init_state._board])
print_strategy(0, 1, 'dd')